In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在 Vertex AI 上部署和托管一个稳定的Diffusion模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/torchserve/dreambooth_stablediffusion.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/torchserve/dreambooth_stablediffusion.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
 <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/torchserve/dreambooth_stablediffusion.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何在Vertex AI上部署和托管经过微调的[稳定扩散1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5)模型。对于托管，您可以使用为Vertex AI构建的带有[TorchServe](https://pytorch.org/serve/index.html)的PyTorch 3容器。

### 目标

在本教程中，您将学习如何在Vertex AI上托管和部署一个稳定的扩散1.5模型。

本教程使用以下Google Cloud ML服务：

+ Vertex AI `Model`资源
+ Vertex AI `Endpoint`资源

执行的步骤包括：

+ 创建一个`torchserve`处理程序来响应预测请求。
+ 在Vertex AI的预构建PyTorch容器中上传一个稳定的扩散1.5模型。
+ 将模型部署到Vertex AI Endpoint。
+ 发送请求到端点，并使用Vertex AI预测服务解析响应。

### 模型

这个笔记本使用了一个经过微调的模型工件集合来生成小狗的图像。这些图像和原始的[DreamBooth论文](https://dreambooth.github.io/)中使用的相同。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI模型
* Vertex AI端点
* Vertex AI预测
* 云存储
* （可选）Vertex AI工作台

了解有关[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)的信息，并使用[定价计算器](https://cloud.google.com/products/calculator/)基于您的预期使用情况生成成本估算。

硬件要求

这个笔记本需要你使用具有足够可用VRAM的GPU。 它在具有85 GB VRAM的`NVIDIA Tesla A100 GPU`上进行了测试。 运行以下单元格以确保您拥有正确的硬件配置。

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format="csv,noheader"

在Vertex AI上创建一个用户管理的笔记本

如果您正在使用Vertex AI Workbench，您可以通过以下方式创建一个具有正确配置的笔记本：

+ 在Google Cloud控制台中前往[Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/user-managed)。
+ 点击**新建笔记本**，然后点击**PyTorch 1.13** > **使用1个NVIDIA T4**。
+ 在**新建笔记本**对话框中，点击**高级选项**。**创建用户管理的笔记本**页面将打开。
+ 在**创建用户管理的笔记本**页面中，执行以下操作：
  * 在**笔记本名称**框中，为您的笔记本输入一个名称，例如"my-stablediffusion-nb"。
  * 在**机器类型**下拉菜单中，选择**A2 highgpu** > **a2-highgpu-1g**。
  * 在**GPU类型**下拉菜单中，选择**NVIDIA Tesla A100**。
  * 勾选**自动为我安装NVIDIA GPU驱动程序**旁边的复选框。
  * 展开**磁盘**并执行以下操作：
    - 在**启动磁盘类型**下，选择**SSD持久性磁盘**。
    - 在**数据磁盘类型**下，选择**SSD持久性磁盘**。
  * 点击**创建**。

<div style="background:#feefe3; padding:5px; color:#aa0000">
<strong>注意：</strong> 使用具有上述配置的Vertex AI Workbench笔记本可能会显著增加您的成本。您可以使用<a href="https://cloud.google.com/products/calculator"><u>成本计算器</u></a>来估算您的成本。</div>

## 安装

安装以下必须的软件包以执行这个笔记本。

**注意**：您可能需要通过 `pip` 修改安装的 PyTorch (`torch`) 版本。

In [ ]:
%%writefile requirements.txt
diffusers
ftfy
google-cloud-aiplatform
gradio
ninja
tensorboard==1.15.0
torch
torchaudio
torchvision
torchserve
torch-model-archiver
torch-workflow-archiver
transformers

In [ ]:
%pip install -r requirements.txt

### 仅限Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要执行以下步骤。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建账户时，您将获得$300的免费信用额，用于支付计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于[Vertex AI的区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动验证。请按照下面的相关说明操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账号授予云存储权限。

导入库

In [ ]:
import base64
import math

import torch
from diffusers import StableDiffusionPipeline
from google.cloud import aiplatform
from IPython import display
from PIL import Image
from torch import autocast

## 可选：查看模型推论

在将模型上传至Vertex AI之前，您可以查看模型的预期输出。此笔记本中使用的模型可供您使用，并可从云存储中下载。此下载可能需要几分钟时间才能完成。

In [ ]:
!gsutil -m cp gs://cloud-samples-data/vertex-ai/model-deployment/models/stable-diffusion/model_artifacts.zip \
   .

In [ ]:
!unzip model_artifacts.zip

### 创建新图像

一切就绪后，您现在可以从 Stable Diffusion 模型中生成新图像。首先，您必须将您的模型加载到一个 `StableDiffusionPipeline` 中。

In [ ]:
model_path = "model_artifacts"

pipe = StableDiffusionPipeline.from_pretrained(
    model_path, torch_dtype=torch.float16
).to("cuda")

g_cuda = None

In [ ]:
g_cuda = torch.Generator(device="cuda")
seed = 52362
g_cuda.manual_seed(seed)

将模型加载到`StableDiffusionPipeline`中后，您现在可以从模型生成结果（推断）。每组推断都需要一个输入（称为[prompt](https://learnprompting.org/)），指定模型应该创建什么。

您还可以更改模型的其他输入，如下面的代码单元所示。

In [ ]:
prompt = "photo of examplePup dog in a Monet style"

num_samples = 4
num_batches = 1
num_columns = 2
guidance_scale = 10
num_inference_steps = 50
height = 512
width = 512

In [ ]:
def image_grid(imgs, cols):
    total = len(imgs)
    rows = math.ceil(total / cols)

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


all_images = []
for _ in range(num_batches):
    with autocast("cuda"):
        images = pipe(
            [prompt] * num_samples,
            height=height,
            width=width,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
        ).images
        all_images.extend(images)


grid = image_grid(all_images, num_columns)
grid

将模型部署到 Vertex AI

您可以将您的 Stable Diffusion 1.5 模型部署到 Vertex AI 端点上，从而可以在线获取推断。上传您的模型是一个四步骤的过程：

1. 创建一个自定义 TorchServe 处理程序。
2. 将模型工件上传到云存储。
3. 使用模型工件和预构建的 PyTorch 容器映像创建一个 Vertex AI 模型。
4. 将 Vertex AI 模型部署到一个端点。

### 创建自定义的TorchServe处理程序

部署到Vertex AI的模型使用[TorchServe](https://pytorch.org/serve/)来处理请求并从模型返回响应。您必须创建一个自定义的TorchServe处理程序，将其与上传到Vertex AI的模型工件一起包含在内。

处理程序文件应包含在与其他模型工件一起的目录中。

In [ ]:
%%writefile model_artifacts/handler.py

"""Customized handler for Stable Diffusion 1.5."""
import base64
import logging
from io import BytesIO

import torch
from diffusers import EulerDiscreteScheduler
from diffusers import StableDiffusionPipeline
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)
model_id = 'runwayml/stable-diffusion-v1-5'


class ModelHandler(BaseHandler):

  def __init__(self):
    self.initialized = False
    self.map_location = None
    self.device = None
    self.use_gpu = True
    self.store_avg = True
    self.pipe = None

  def initialize(self, context):
    """Initializes the pipe."""
    properties = context.system_properties
    gpu_id = properties.get('gpu_id')

    self.map_location, self.device, self.use_gpu = \
      ('cuda', torch.device('cuda:' + str(gpu_id)),
       True) if torch.cuda.is_available() else \
        ('cpu', torch.device('cpu'), False)

    # Use the Euler scheduler here instead
    scheduler = EulerDiscreteScheduler.from_pretrained(model_id,
                                                       subfolder='scheduler')
    pipe = StableDiffusionPipeline.from_pretrained(model_id,
                                                   scheduler=scheduler,
                                                   torch_dtype=torch.float16)
    pipe = pipe.to('cuda')
    # Uncomment the following line to reduce the GPU memory usage.
    # pipe.enable_attention_slicing()
    self.pipe = pipe

    self.initialized = True

  def preprocess(self, requests):
    """Noting to do here."""
    logger.info('requests: %s', requests)
    return requests

  def inference(self, preprocessed_data, *args, **kwargs):
    """Run the inference."""
    images = []
    for pd in preprocessed_data:
      prompt = pd['prompt']
      images.extend(self.pipe(prompt).images)
    return images

  def postprocess(self, output_batch):
    """Converts the images to base64 string."""
    postprocessed_data = []
    for op in output_batch:
      fp = BytesIO()
      op.save(fp, format='JPEG')
      postprocessed_data.append(base64.b64encode(fp.getvalue()).decode('utf-8'))
      fp.close()
    return postprocessed_data


创建处理程序文件后，您必须将处理程序打包为模型归档器（MAR）文件。输出文件必须命名为'model.mar'。

In [ ]:
!torch-model-archiver \
  -f \
  --model-name model \
  --version 1.0 \
  --handler model_artifacts/handler.py \
  --export-path model_artifacts

### 将模型制品上传至云存储

在您的云存储桶中创建一个新文件夹来保存模型制品。

In [ ]:
BUCKET_NAME = "your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}/"
FULL_GCS_PATH = f"{BUCKET_URI}model_artifacts"

只有当您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

接下来，将模型存档文件和您训练好的 Stable Diffusion 1.5 模型上传到云存储上的文件夹中。

In [ ]:
!gsutil cp -r model_artifacts $BUCKET_URI

### 创建 Vertex AI 模型

一旦您将模型工件上传到云存储桶中，您就可以创建一个新的 Vertex AI 模型。这个笔记本使用 [Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/use-vertex-ai-python-sdk) 来创建模型。

In [ ]:
PYTORCH_PREDICTION_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.1-12:latest"
)
APP_NAME = "my-stable-diffusion"
VERSION = 1
MODEL_DISPLAY_NAME = "stable_diffusion_1_5-unique"
MODEL_DESCRIPTION = "stable_diffusion_1_5 container"
ENDPOINT_DISPLAY_NAME = f"{APP_NAME}-endpoint"

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

请注意：如果您尚未 <a href="https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com"><u>启用了Vertex API</u></a>，则下一个单元格将失败。</div>

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    description=MODEL_DESCRIPTION,
    serving_container_image_uri=PYTORCH_PREDICTION_IMAGE_URI,
    artifact_uri=FULL_GCS_PATH,
)

model.wait()

print(model.display_name)
print(model.resource_name)

### 将模型部署到端点

要从您的 Stable Diffusion 2.0 模型获取在线预测，您必须[将其部署到 Vertex AI 端点](https://cloud.google.com/vertex-ai/docs/predictions/overview)。您可以再次使用 Vertex AI SDK 创建端点并部署您的模型。

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

In [ ]:
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_DISPLAY_NAME,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_P100",
    accelerator_count=1,
    traffic_percentage=100,
    deploy_request_timeout=1200,
    sync=True,
)

之前的单元格会花一些时间来完成部署模型到端点的操作。如果之前的单元格在返回之前超时了，你的端点可能仍然成功地部署到了端点。检查[Cloud控制台](https://console.cloud.google.com/vertex-ai/endpoints)来验证结果。

你也可以通过更改传递给`model.deploy()`的`deploy_request_timeout`参数来延长等待部署的时间。

获得在线预测

最终，将您的Stable Diffusion 1.5模型部署到Vertex AI端点后，您现在可以从中获得在线预测。使用Vertex AI SDK，您只需几行代码就能得出推断。

In [ ]:
instances = [{"prompt": "An examplePup dog with a baseball jersey."}]
response = endpoint.predict(instances=instances)

with open("img5.jpg", "wb") as g:
    g.write(base64.b64decode(response.predictions[0]))

In [ ]:
display.Image("img5.jpg")

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

# Delete endpoint resource
endpoint.undeploy_all()
endpoint.delete()

# Delete model resource
model.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI